In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 0. Configuration

In [ ]:
MODEL_NAME = "google/flan-t5-large"

PATH_BASE = "/content/drive/My Drive/"
PATH_CLAIM_DECOMP = PATH_BASE + "ClaimDecomp/"
PATH_CLAIM_DECOMP_TRAIN = PATH_CLAIM_DECOMP + "train.jsonl"
PATH_CLAIM_DECOMP_VAL = PATH_CLAIM_DECOMP + "dev.jsonl"
PATH_CLAIM_DECOMP_TEST = PATH_CLAIM_DECOMP + "test.jsonl"
PATH_DEEP_SPEED_CONFIG = PATH_CLAIM_DECOMP + "ds_config.json"

LEARNING_RATE = 3e-5
BATCH_SIZE = 4
NUM_EPOCHS = 20

# 1. Install dependencies

In [ ]:
!pip install datasets transformers rouge_score
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b48adb12558e1e3182a3709894eca719be7d42d0e298bb80d09ef25d5d867510
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages

# 2. Load model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# 3. Load dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files={"train": PATH_CLAIM_DECOMP_TRAIN, "val": PATH_CLAIM_DECOMP_VAL, "test": PATH_CLAIM_DECOMP_TEST})
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['example_id', 'label', 'url', 'annotations', 'claim'],
        num_rows: 800
    })
    val: Dataset({
        features: ['example_id', 'label', 'url', 'annotations', 'claim'],
        num_rows: 200
    })
    test: Dataset({
        features: ['example_id', 'label', 'url', 'annotations', 'claim'],
        num_rows: 200
    })
})

In [ ]:
print(dataset)
# print(dataset['train']['url'][3])
print(dataset['test']['claim'][4])
print(dataset['test']['annotations'][0][0]['statements'])
print(dataset['test']['annotations'][0][1]['statements'])

# print(dataset['train']['annotations'][5][0])
# print(dataset['train']['annotations'][0][0]['statements'])

DatasetDict({
    train: Dataset({
        features: ['example_id', 'label', 'url', 'annotations', 'claim'],
        num_rows: 800
    })
    val: Dataset({
        features: ['example_id', 'label', 'url', 'annotations', 'claim'],
        num_rows: 200
    })
    test: Dataset({
        features: ['example_id', 'label', 'url', 'annotations', 'claim'],
        num_rows: 200
    })
})
"The president has said the national security threat facing America, the top one, is global warming."
[' Barr has received $36,550 from payday lenders.', ' Barr voted for legislation that would weaken restrictions for payday lenders.', ' There are protections for service members using payday lending services.', " Barr's voting record has directly affected protection for veterans against payday lenders."]
[' Bar accepted $36,000 from pay day lenders.', ' Bar does not let pay day lenders take advantage of troops.']


# 4. Preprocess data

In [ ]:
# def generate_prompt(claim):
#   return "Decompose the following claim into three subquestions, which need to be answered before the veracity of the original claim can be deduced: " + claim + ".\nOutput the subquestions seperated with comma's."

# def preprocess_function(examples):
#     inputs = [generate_prompt(claim) for claim in examples['claim']]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
#     sub_claims = ['\n'.join([f"{i + 1}. " + b.strip() for i, b in enumerate(a[0]['statements'])]) for a in examples['annotations']]
#     labels = tokenizer(text_target=sub_claims, max_length=512, padding='max_length')
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

def preprocess_function(examples):
  inputs = []
  targets = []
  for i in range(len(examples['claim'])):
    i_annotions = 0 if len(examples['annotations'][i][0]) >= len(examples['annotations'][i][1]) else 1
    sub_questions = examples['annotations'][i][i_annotions]['questions']
    n = len(sub_questions)
    claim = examples['claim'][i]

    inputs.append(str(len(sub_questions)) + ' # ' + claim)
    targets.append(' '.join(sub_questions))
  print(inputs)
  print(targets)
  model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
  labels = tokenizer(text_target=targets, max_length=256, padding='max_length')
  labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_train = dataset['train'].map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
tokenized_val = dataset['val'].map(preprocess_function, batched=True, remove_columns=dataset['val'].column_names)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

['3 # "Unemployment is low because everyone has two jobs. Unemployment is low because people are working 60, 70, 80 hours a week and can barely feed their family."', '3 # Says the impeachment of President Donald Trump would be the first in U.S. history "without a specific criminal statute or crime that the president who is impeached would have allegedly committed."', '3 # "When I withdrew in June of 2008, polls were showing that at least 40 percent of my supporters said, oh, they weren\'t going to support Sen. Obama."', '3 # Text from someone “connected” to Raleigh police: “antifa may be heading into the wealthy neighborhoods tonight to take what should be theirs.”', '3 # “Some states, like Montana and Nebraska, are getting more than $300,000 in federal stimulus money per reported COVID-19 case. New York is the hardest-hit state and yet we are getting only about $12,000 per case.”', '2 # "Trey Radel does not even qualify to drive a Lee County school bus at this point, yet he occupies a

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

["3 # With voting by mail, “you get thousands and thousands of people sitting in somebody's living room, signing ballots all over the place.”", '3 # "I’ve already traveled to Washington, D.C., and secured a historic commitment from the Trump administration to provide assistance to the communities that Michael battered."', '3 # Says that when San Francisco banned plastic grocery bags, "you saw the number of instances of people going to the ER with things like salmonella and other related illnesses" spike.', '1 # The United States "decided waterboarding was torture back when we court-martialed American soldiers for waterboarding Philippine insurgents in the Philippine revolution."', '3 # Quotes Donald Trump as saying, “I’ll tell you the funniest thing. I’ll go backstage before a Miss Teen USA show, and everyone’s getting dressed and ready and everything and no men are anywhere. … You know, I’m inspecting. The girls (14-16) are standing there with no clothes on.”', '3 # Says an election c

In [ ]:
t10 = tokenized_train.select(range(10))
v10 = tokenized_val.select(range(10))

In [ ]:
t16

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16
})

In [ ]:
print(' '.join(tokenizer.batch_decode(tokenized_train['input_ids'][0])))
print(' '.join(tokenizer.batch_decode(np.where(np.array(tokenized_train['labels'][0]) != -100, tokenized_train['labels'][0], tokenizer.pad_token_id))))
# print(' '.join(tokenizer.batch_decode()
# print(tokenized_train['labels'][0])


3 </s> " Un employment is low because everyone has two jobs . Un employment is low because people are working 60 , 70 , 80 hours  a week and can barely feed their family ." </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Can low unemployment rates be  attributed to everyone having 2 jobs ? </s> I s unemployment even currently low because of factors stated by O cas i o Cor t ez ? </s> Can low unemployment be  attributed to long work hours ? </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

# 4. Prepare training

In [ ]:
from datasets import load_metric
import numpy as np

# Metric
metric_name = "rouge"
metric = load_metric(metric_name)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    print(decoded_preds)
    print(decoded_labels)

    if metric_name == "bleu":
        decoded_labels = [[label.split()] for label in decoded_labels]
        decoded_preds = [pred.split() for pred in decoded_preds]

    # for pred, lable in zip(decoded_preds, decoded_labels):
    #     print(pred)
    #     print(lable)
    #     input()

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    # print(result)
    # Extract a few results from ROUGE
    if metric_name == "rouge":
        result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    else:
        result = {"bleu": result["bleu"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    return result

<ipython-input-8-e6c9b97bd862>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric(metric_name)
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
import torch

# # Step 4: Set up training arguments
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=LEARNING_RATE,
#     per_device_train_batch_size=BATCH_SIZE,
#     per_device_eval_batch_size=BATCH_SIZE,
#     num_train_epochs=NUM_EPOCHS,
#     predict_with_generate=True,
#     weight_decay=0.01,
#     save_steps=500,
#     save_total_limit=3,                    # Only last 3 models are saved
#     # fp16=True,
#     optim="adamw_torch",             # Use AdamW optimizer
#     lr_scheduler_type="linear",      # Linear scheduler
#     # gradient_accumulation_steps=2
#     generation_max_length=256
# )

# Step 4: Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    weight_decay=0.01,
    # save_steps=500,
    save_total_limit=3,
    # fp16=True,
    # optim="adamw_torch",             # Use AdamW optimizer
    # lr_scheduler_type="linear",      # Linear scheduler
    # # gradient_accumulation_steps=2
    generation_max_length=256
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100, pad_to_multiple_of=8)

# Step 5: Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 5. Train

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.600504,40.691124,20.745522,36.121400,36.217115,38.650000
2,No log,1.563733,41.381384,21.900399,37.461760,37.480539,42.170000
3,1.723200,1.560451,41.936295,22.138074,37.832626,37.879101,47.530000
4,1.723200,1.561853,42.642658,22.796027,38.049503,38.098191,45.165000
5,1.359300,1.578908,42.623803,22.646321,37.815627,37.883234,45.870000
6,1.359300,1.593772,42.707904,22.760555,37.844220,37.904042,46.445000
7,1.359300,1.620400,43.079559,22.807427,37.973930,38.023187,46.175000
8,1.119800,1.647872,43.098667,23.197602,37.505816,37.640434,46.860000
9,1.119800,1.670937,43.214094,23.039772,37.418854,37.490795,46.970000
10,0.970700,1.705973,42.530728,22.302893,36.751661,36.845263,46.805000


['Do people sign ballots in their living rooms? Do people sign ballots in their living rooms? Do people sign ballots in their living rooms?', 'Did Trump agree to provide assistance to communities that Michael battered? Did Trump agree to provide assistance to communities that Michael battered? Did Trump agree to provide assistance to communities that Michael battered?', 'Did the number of instances of people going to the ER with things like salmonella and other related illnesses spike when San Francisco banned plastic grocery bags? Did the number of instances of people going to the ER with things like salmonella and other related illnesses spike when San Francisco banned plastic grocery bags?', 'Did the United States decide waterboarding was torture back when we court-martialed American soldiers for waterboarding Philippine insurgents in the Philippine revolution?', 'Did Trump say this in a statement? Did Trump say this in a statement? Did Trump say this in a statement?', 'Did the elec

TrainOutput(global_step=4000, training_loss=1.026251510620117, metrics={'train_runtime': 6841.312, 'train_samples_per_second': 2.339, 'train_steps_per_second': 0.585, 'total_flos': 9219071803392000.0, 'train_loss': 1.026251510620117, 'epoch': 20.0})

In [ ]:
trainer.save_model("/content/drive/My Drive/t5_large_claim_decomp")

# 6. Eval

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/big_boi')

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluation results: {'eval_loss': 38.46904754638672, 'eval_f1_weighted': 0.3205, 'eval_f1_macro': 0.2641, 'eval_runtime': 288.144, 'eval_samples_per_second': 10.703, 'eval_steps_per_second': 2.676}


In [ ]:
device = torch.device('cpu')
model.to(device)

print(dataset['val']['claim'][1])
print(generate_prompt(dataset['val']['claim'][1]))
inputs = generate_prompt(dataset['val']['claim'][1])
model_inputs = tokenizer(inputs, return_tensors="pt")
outputs = model.generate(**model_inputs, max_new_tokens=512)
model_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(model_output)

"I’ve already traveled to Washington, D.C., and secured a historic commitment from the Trump administration to provide assistance to the communities that Michael battered."
Decompose the following claim into three subquestions, which need to be answered before the veracity of the original claim can be deduced: "I’ve already traveled to Washington, D.C., and secured a historic commitment from the Trump administration to provide assistance to the communities that Michael battered."
['What is the Trump administration’s commitment to the communities that Michael battered?']


In [ ]:
import pandas as pd
from datasets import Dataset
from ast import literal_eval

BASE_LOCATION = "/content/drive/My Drive/"
TRAIN_LOCATION = BASE_LOCATION + "train_top_20.csv"
VAL_LOCATION = BASE_LOCATION + "val_top_20.csv"

train_dataset = pd.read_csv(TRAIN_LOCATION, index_col=0)
train_dataset['evidences'] = train_dataset['evidences'].apply(literal_eval)
train_dataset['scores'] = train_dataset['scores'].apply(literal_eval)
train_dataset = Dataset.from_pandas(train_dataset)

val_dataset = pd.read_csv(VAL_LOCATION, index_col=0)
val_dataset['evidences'] = val_dataset['evidences'].apply(literal_eval)
val_dataset['scores'] = val_dataset['scores'].apply(literal_eval)
val_dataset = Dataset.from_pandas(val_dataset)

In [ ]:
def generate_prompt(claim, evidences):
    prompt_begin = (
        f"You are a fact-checker. Your task is to classify the claim based on the following evidence.\n\n"
        f"Claim: {claim}\n\n"
        f"Evidences:\n"
    )
    prompt_evidences = f""
    prompt_end = f"\n\nLabel (TRUE, FALSE, CONFLICTING):"

    for k in range(0, min(5, len(evidences))):
        new_evidence = f"\n{k + 1}. {evidences[k][0]} (score: {round(evidences[k][1], 2)})"
        tokens = tokenizer(prompt_begin + prompt_evidences + new_evidence + prompt_end, max_length=512, truncation=True, return_tensors="pt")
        if tokens['input_ids'].shape[1] < tokenizer.model_max_length:
            prompt_evidences += new_evidence
        else:
            break
    return prompt_begin + prompt_evidences + prompt_end

def preprocess_function(examples):
    inputs = [generate_prompt(examples['claim'][i], list(zip(examples['evidences'][i], examples['scores'][i]))) for i in range(len(examples["claim"]))]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["label"], max_length=16, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/9935 [00:00<?, ? examples/s]

Map:   0%|          | 0/3084 [00:00<?, ? examples/s]

In [ ]:
t16 = tokenized_train.select(range(16))
v16 = tokenized_validation.select(range(16))